In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# Загрузка данных из CSV-файла
telegram_messages = pd.read_csv('telegram_messages.csv')

# Преобразование столбца 'date' в формат datetime
telegram_messages['date'] = pd.to_datetime(telegram_messages['date'])

# Загрузка модели и токенизатора
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Создание пайплайна для анализа тональности
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Функция для обрезки текста до 512 токенов
def truncate_text(text, tokenizer, max_length=500):
    tokens = tokenizer.tokenize(text)
    if len(tokens) > max_length:
        tokens = tokens[:max_length]
    return tokenizer.convert_tokens_to_string(tokens)

# Функция для получения тональности
def get_bert_sentiment(text):
    truncated_text = truncate_text(text, tokenizer)
    result = sentiment_analysis(truncated_text)[0]
    return result['label']

# Применение анализа тональности к сообщениям
telegram_messages['sentiment'] = telegram_messages['message'].apply(lambda x: get_bert_sentiment(str(x)) if pd.notna(x) else 'neutral')

# Подсчет количества сообщений с каждой тональностью
sentiment_counts = telegram_messages['sentiment'].value_counts()
print(f"Сентимент анализа:\n{sentiment_counts}")

In [ ]:
telegram_messages.to_csv('data/telegram_mes sages_sentiment.csv', index=False)